In [1]:
pip install redshift-connector


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for redshift-connector from https://files.pythonhosted.org/packages/52/2d/d1402bd60e2536db201146c81d09fd8a7f895001aac1bcadb8d5662ca3f0/redshift_connector-2.0.918-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/66.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/66.2 kB ? eta -:--:--
     ----------------------- -------------- 41.0/66.2 kB 487.6 kB/s eta 0:00:01
     ----------------------------------- -- 61.4/66.2 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 66.2/66.2 kB 447.4 kB/s eta 0:00:00
  Obtaining dependency information for lxml>=4.6.5 from https://files.pythonhosted.org/packages/5b/d6/e794d996dec76b773691af78390fe2f419ab7cb5b78a4df982e21ae655b7/lxml-5.1.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/105.0 kB ? eta -:--:--
     -------------------------------------- 105.0/105.0 kB 3.1 MB/s eta 0:00:00
   --------------------------

In [5]:
import boto3
import pandas as pd
from io import StringIO

In [7]:
from datetime import datetime

In [8]:
import numpy as np

In [9]:
AWS_ACCESS_KEY = '--'
AWS_SECRET_KEY = '--'
AWS_REGION = 'us-east-1'
SCHEMA_NAME = 'financials_20years'
S3_STAGING_DIR = 's3://financialanalysis/financial-analysis/'
S3_BUCKET_NAME = 'financialanalysis'
S3_OUTPUT_DIR = 'financial-analysis'

In [10]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [11]:
Dict={}
def download_and_load_query_result(client: boto3.client, query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId = query_response['QueryExecutionId']
            )
            break
        except Exception as e:
            if "not yet finished" in str(e):
                time.sleep(0.001)
            else:
                raise(e)
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
    )

    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)
    

## AAPL Extraction

In [12]:
response_aapl = athena_client.start_query_execution(
    QueryString = "SELECT * FROM aapl_d8b9221a1d13a740339e56110094582b",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [13]:
AAPL = download_and_load_query_result(athena_client, response_aapl)

In [14]:
AAPL

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,132.7600,134.80,130.53,134.18,134.1800,152470142,0.0,1.0
1,2020-08-31,127.5800,131.00,126.00,129.04,129.0400,223505733,0.0,4.0
2,2020-08-28,504.0500,505.77,498.31,499.23,124.8075,46907479,0.0,1.0
3,2020-08-27,508.5700,509.94,495.33,500.04,125.0100,38888096,0.0,1.0
4,2020-08-26,504.7165,507.97,500.33,506.09,126.5225,40755567,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,96.5000,101.00,95.50,99.50,0.7676,4113700,0.0,1.0
5196,2000-01-06,106.1200,107.00,95.00,95.00,0.7329,6856900,0.0,1.0
5197,2000-01-05,103.7500,110.56,103.00,104.00,0.8023,6949300,0.0,1.0
5198,2000-01-04,108.2500,110.62,101.19,102.50,0.7907,4574800,0.0,1.0


## AMZN Extraction

In [15]:
response_amzn = athena_client.start_query_execution(
    QueryString = "SELECT * FROM amzn_e6deb17ade85c1f765ead482a6b739f1",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [17]:
AMZN = download_and_load_query_result(athena_client, response_amzn)

In [18]:
AMZN

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,3489.58,3513.8700,3467.0000,3499.12,3499.12,3476407,0.0,1.0
1,2020-08-31,3408.99,3495.0000,3405.0000,3450.96,3450.96,4185885,0.0,1.0
2,2020-08-28,3423.00,3433.3699,3386.5000,3401.80,3401.80,2896978,0.0,1.0
3,2020-08-27,3450.05,3453.0000,3378.0000,3400.00,3400.00,4264795,0.0,1.0
4,2020-08-26,3351.11,3451.7387,3344.5675,3441.85,3441.85,6508743,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,67.00,70.5000,66.1900,69.56,69.56,10505400,0.0,1.0
5196,2000-01-06,71.31,72.6900,64.0000,65.56,65.56,18752000,0.0,1.0
5197,2000-01-05,70.75,75.1300,69.6300,71.75,71.75,38457400,0.0,1.0
5198,2000-01-04,85.38,91.5000,81.7500,81.94,81.94,17487400,0.0,1.0


## BRK-A Extraction

In [19]:
response_brka = athena_client.start_query_execution(
    QueryString = "SELECT * FROM brk_a_428d833527d40a7732b26fb87d439e23",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [20]:
BRKA = download_and_load_query_result(athena_client, response_brka)

In [21]:
BRKA

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,325500.0,327560.00,325335.60,327440.0,327440.0,342,0.0,1.0
1,2020-08-31,327500.0,329299.19,326900.00,327560.0,327560.0,454,0.0,1.0
2,2020-08-28,325950.0,328279.19,323570.01,327431.0,327431.0,299,0.0,1.0
3,2020-08-27,323700.0,327000.00,323700.00,325659.0,325659.0,578,0.0,1.0
4,2020-08-26,319599.9,323939.46,318900.00,322301.0,322301.0,429,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,55600.0,56500.00,55200.00,56500.0,56500.0,67000,0.0,1.0
5196,2000-01-06,53300.0,55000.00,53100.00,55000.0,55000.0,57000,0.0,1.0
5197,2000-01-05,51700.0,54700.00,51700.00,53200.0,53200.0,51000,0.0,1.0
5198,2000-01-04,53700.0,53800.00,52000.00,52000.0,52000.0,44000,0.0,1.0


## FB Extraction

In [22]:
response_fb = athena_client.start_query_execution(
    QueryString = "SELECT * FROM fb_7ba340d9cd32b21b9e6075654bd4b140",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [23]:
FB = download_and_load_query_result(athena_client, response_fb)
FB

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,294.71,301.49,292.71,295.4400,295.4400,17320872,0.0,1.0
1,2020-08-31,293.95,296.88,291.55,293.2000,293.2000,17345134,0.0,1.0
2,2020-08-28,295.00,297.23,290.98,293.6600,293.6600,17172415,0.0,1.0
3,2020-08-27,300.16,301.23,292.02,293.2200,293.2200,30301309,0.0,1.0
4,2020-08-26,284.00,304.67,284.00,303.9100,303.9100,69015151,0.0,1.0
...,...,...,...,...,...,...,...,...,...
2081,2012-05-24,32.95,33.21,31.77,33.0300,33.0300,50237200,0.0,1.0
2082,2012-05-23,31.37,32.50,31.36,32.0000,32.0000,73600000,0.0,1.0
2083,2012-05-22,32.61,33.59,30.94,31.0000,31.0000,101786600,0.0,1.0
2084,2012-05-21,36.53,36.66,33.00,34.0300,34.0300,168192700,0.0,1.0


## GOOG Extraction

In [24]:
response_goog = athena_client.start_query_execution(
    QueryString = "SELECT * FROM goog",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [25]:
GOOG = download_and_load_query_result(athena_client, response_goog)
GOOG

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,1636.630,1665.7300,1632.22,1660.71,1660.71,1826655,0.0,1.0
1,2020-08-31,1647.890,1647.9650,1630.31,1634.18,1634.18,1823406,0.0,1.0
2,2020-08-28,1633.490,1647.1699,1630.75,1644.41,1644.41,1499782,0.0,1.0
3,2020-08-27,1653.680,1655.0000,1625.75,1634.33,1634.33,1861602,0.0,1.0
4,2020-08-26,1608.000,1659.2200,1603.60,1652.38,1652.38,3581072,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1616,2014-04-02,565.106,604.8300,562.19,567.00,567.00,146700,0.0,1.0
1617,2014-04-01,558.710,568.4500,558.71,567.16,567.16,7900,0.0,1.0
1618,2014-03-31,566.890,567.0000,556.93,556.97,556.97,10800,0.0,1.0
1619,2014-03-28,561.200,566.4300,558.67,559.99,559.99,41100,0.0,1.0


## JNJ Extraction

In [26]:
response_jnj = athena_client.start_query_execution(
    QueryString = "SELECT * FROM jnj",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [27]:
JNJ = download_and_load_query_result(athena_client, response_jnj)
JNJ

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,153.87,154.08,150.0700,151.52,151.5200,6309237,0.0,1.0
1,2020-08-31,153.00,154.40,152.2605,153.41,153.4100,6362683,0.0,1.0
2,2020-08-28,152.74,153.70,151.9000,153.64,153.6400,5395684,0.0,1.0
3,2020-08-27,152.42,153.26,151.8900,152.98,152.9800,5575432,0.0,1.0
4,2020-08-26,151.86,152.65,150.6400,152.30,152.3000,8008292,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,94.13,96.94,93.6300,96.50,28.5249,4931900,0.0,1.0
5196,2000-01-06,90.31,93.94,90.1300,92.56,27.3602,2678200,0.0,1.0
5197,2000-01-05,88.81,90.81,88.4400,90.19,26.6597,4114800,0.0,1.0
5198,2000-01-04,91.13,91.56,88.7500,88.81,26.2518,3911100,0.0,1.0


## MA Extraction

In [28]:
response_ma = athena_client.start_query_execution(
    QueryString = "SELECT * FROM ma",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [29]:
MA = download_and_load_query_result(athena_client, response_ma)
MA

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,357.71,359.44,353.5600,356.90,356.9000,4412230,0.0,1.0
1,2020-08-31,366.63,366.63,357.6898,358.19,358.1900,3948240,0.0,1.0
2,2020-08-28,357.48,367.25,356.1600,366.12,366.1200,3544720,0.0,1.0
3,2020-08-27,351.73,358.00,350.5800,356.00,356.0000,3811724,0.0,1.0
4,2020-08-26,348.00,352.29,345.5000,351.02,351.0200,3511970,0.0,1.0
...,...,...,...,...,...,...,...,...,...
3588,2006-06-01,44.93,48.10,44.9000,47.51,4.4471,6234400,0.0,1.0
3589,2006-05-31,44.35,45.36,44.3500,44.94,4.2065,3000200,0.0,1.0
3590,2006-05-30,44.97,44.98,42.8500,44.00,4.1185,4989800,0.0,1.0
3591,2006-05-26,46.31,46.74,44.1100,44.93,4.2056,10304400,0.0,1.0


## MSFT Extraction

In [30]:
response_msft = athena_client.start_query_execution(
    QueryString = "SELECT * FROM msft",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [31]:
MSFT = download_and_load_query_result(athena_client, response_msft)
MSFT

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,225.51,227.450,224.43,227.27,227.2700,25791235,0.0,1.0
1,2020-08-31,227.00,228.700,224.31,225.53,225.5300,28774156,0.0,1.0
2,2020-08-28,228.18,230.644,226.58,228.91,228.9100,26292896,0.0,1.0
3,2020-08-27,222.89,231.150,219.40,226.58,226.5800,57602195,0.0,1.0
4,2020-08-26,217.88,222.090,217.36,221.15,221.1500,39600828,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,108.62,112.250,107.31,111.44,35.6435,31006800,0.0,1.0
5196,2000-01-06,112.19,113.870,108.37,110.00,35.1829,27488300,0.0,1.0
5197,2000-01-05,111.12,116.370,109.37,113.81,36.4015,32029800,0.0,1.0
5198,2000-01-04,113.56,117.120,112.25,112.62,36.0209,27059500,0.0,1.0


## V Extraction

In [32]:
response_v = athena_client.start_query_execution(
    QueryString = "SELECT * FROM v",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [33]:
V = download_and_load_query_result(athena_client, response_v)
V

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,212.21,213.89,211.1200,213.35,213.3500,5648766,0.0,1.0
1,2020-08-31,215.54,215.96,211.5600,211.99,211.9900,9326852,0.0,1.0
2,2020-08-28,212.00,216.16,211.2200,215.71,215.7100,7349837,0.0,1.0
3,2020-08-27,211.21,212.18,209.0500,211.03,211.0300,8619578,0.0,1.0
4,2020-08-26,208.10,210.80,207.3025,210.26,210.2600,7035221,0.0,1.0
...,...,...,...,...,...,...,...,...,...
3132,2008-03-26,62.50,64.48,61.5700,63.96,14.6328,10777900,0.0,1.0
3133,2008-03-25,60.63,64.25,59.8200,63.25,14.4704,21773000,0.0,1.0
3134,2008-03-24,67.24,67.70,59.0000,59.73,13.6651,37391600,0.0,1.0
3135,2008-03-20,58.47,65.00,57.5000,64.35,14.7221,49746300,0.0,1.0


## WMT Extraction

In [34]:
response_wmt = athena_client.start_query_execution(
    QueryString = "SELECT * FROM wmt",
    QueryExecutionContext = { "Database": SCHEMA_NAME },
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": { "EncryptionOption": "SSE_S3" },
    },
    )

In [35]:
WMT = download_and_load_query_result(athena_client, response_wmt)
WMT

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,141.20,149.59,140.05,147.59,147.5900,35708207,0.0,1.0
1,2020-08-31,137.48,138.89,135.53,138.85,138.8500,15078777,0.0,1.0
2,2020-08-28,141.00,141.07,137.84,140.30,140.3000,21350313,0.0,1.0
3,2020-08-27,131.08,139.35,130.90,136.63,136.6300,39672909,0.0,1.0
4,2020-08-26,130.21,130.88,129.70,130.70,130.7000,6271556,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5195,2000-01-07,64.50,68.94,64.50,68.50,46.9577,7976900,0.0,1.0
5196,2000-01-06,63.00,64.56,62.69,63.69,43.6604,6544500,0.0,1.0
5197,2000-01-05,63.88,64.38,62.19,62.81,43.0571,7018700,0.0,1.0
5198,2000-01-04,65.50,65.81,64.19,64.31,44.0854,6745100,0.0,1.0


## Adding the Symbol & Exchange column

In [36]:
AAPL = AAPL.assign(Symbol = 'AAPL')

In [37]:
AMZN = AMZN.assign(Symbol = 'AMZN')

In [38]:
BRKA = BRKA.assign(Symbol = 'BRK-A')

In [39]:
FB = FB.assign(Symbol = 'FB')

In [40]:
GOOG = GOOG.assign(Symbol = 'GOOG')

In [41]:
JNJ = JNJ.assign(Symbol = 'JNJ')

In [42]:
MA = MA.assign(Symbol = 'MA')

In [43]:
MSFT = MSFT.assign(Symbol = 'MSFT')

In [44]:
V = V.assign(Symbol = 'V')

In [45]:
WMT = WMT.assign(Symbol = 'WMT')

In [56]:
list = ['MA', 'BRK-A', 'JNJ', 'V', 'WMT']

In [57]:
df_combine = df_combine.assign(exchange = ['NYSE' if x in list else 'NASDAQ' for x in df_combine['Symbol']])

## Combine ALL the Dataframes

In [48]:
df_combine = pd.concat([AAPL, AMZN, BRKA, FB, GOOG, JNJ, MA, MSFT, V, WMT], ignore_index = True)

In [58]:
df_combine

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,Symbol,day_of_week,day_name,monthName,exchange
0,2020-09-01,132.7600,134.80,130.53,134.18,134.1800,152470142,0.0,1.0,AAPL,1,Tuesday,September,NASDAQ
1,2020-08-31,127.5800,131.00,126.00,129.04,129.0400,223505733,0.0,4.0,AAPL,0,Monday,August,NASDAQ
2,2020-08-28,504.0500,505.77,498.31,499.23,124.8075,46907479,0.0,1.0,AAPL,4,Friday,August,NASDAQ
3,2020-08-27,508.5700,509.94,495.33,500.04,125.0100,38888096,0.0,1.0,AAPL,3,Thursday,August,NASDAQ
4,2020-08-26,504.7165,507.97,500.33,506.09,126.5225,40755567,0.0,1.0,AAPL,2,Wednesday,August,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41632,2000-01-07,64.5000,68.94,64.50,68.50,46.9577,7976900,0.0,1.0,WMT,4,Friday,January,NYSE
41633,2000-01-06,63.0000,64.56,62.69,63.69,43.6604,6544500,0.0,1.0,WMT,3,Thursday,January,NYSE
41634,2000-01-05,63.8800,64.38,62.19,62.81,43.0571,7018700,0.0,1.0,WMT,2,Wednesday,January,NYSE
41635,2000-01-04,65.5000,65.81,64.19,64.31,44.0854,6745100,0.0,1.0,WMT,1,Tuesday,January,NYSE


## Datetime Manipulations

In [50]:
df_combine['timestamp'] = pd.to_datetime(df_combine['timestamp'])

In [51]:
df_combine = df_combine.assign(day_of_week = df_combine['timestamp'].dt.dayofweek)

In [52]:
df_combine = df_combine.assign(day_name = df_combine['day_of_week'])

In [53]:
df_combine['day_name'] = df_combine['day_name'].map({
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
})

In [54]:
df_combine = df_combine.assign(monthName = df_combine['timestamp'].dt.month_name(locale = 'English'))

In [55]:
df_combine

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,Symbol,day_of_week,day_name,monthName
0,2020-09-01,132.7600,134.80,130.53,134.18,134.1800,152470142,0.0,1.0,AAPL,1,Tuesday,September
1,2020-08-31,127.5800,131.00,126.00,129.04,129.0400,223505733,0.0,4.0,AAPL,0,Monday,August
2,2020-08-28,504.0500,505.77,498.31,499.23,124.8075,46907479,0.0,1.0,AAPL,4,Friday,August
3,2020-08-27,508.5700,509.94,495.33,500.04,125.0100,38888096,0.0,1.0,AAPL,3,Thursday,August
4,2020-08-26,504.7165,507.97,500.33,506.09,126.5225,40755567,0.0,1.0,AAPL,2,Wednesday,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41632,2000-01-07,64.5000,68.94,64.50,68.50,46.9577,7976900,0.0,1.0,WMT,4,Friday,January
41633,2000-01-06,63.0000,64.56,62.69,63.69,43.6604,6544500,0.0,1.0,WMT,3,Thursday,January
41634,2000-01-05,63.8800,64.38,62.19,62.81,43.0571,7018700,0.0,1.0,WMT,2,Wednesday,January
41635,2000-01-04,65.5000,65.81,64.19,64.31,44.0854,6745100,0.0,1.0,WMT,1,Tuesday,January


In [59]:
df_combine = df_combine.assign(year = df_combine['timestamp'].dt.year)

In [60]:
df_combine = df_combine.assign(month = df_combine['timestamp'].dt.month)

In [61]:
df_combine

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,Symbol,day_of_week,day_name,monthName,exchange,year,month
0,2020-09-01,132.7600,134.80,130.53,134.18,134.1800,152470142,0.0,1.0,AAPL,1,Tuesday,September,NASDAQ,2020,9
1,2020-08-31,127.5800,131.00,126.00,129.04,129.0400,223505733,0.0,4.0,AAPL,0,Monday,August,NASDAQ,2020,8
2,2020-08-28,504.0500,505.77,498.31,499.23,124.8075,46907479,0.0,1.0,AAPL,4,Friday,August,NASDAQ,2020,8
3,2020-08-27,508.5700,509.94,495.33,500.04,125.0100,38888096,0.0,1.0,AAPL,3,Thursday,August,NASDAQ,2020,8
4,2020-08-26,504.7165,507.97,500.33,506.09,126.5225,40755567,0.0,1.0,AAPL,2,Wednesday,August,NASDAQ,2020,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41632,2000-01-07,64.5000,68.94,64.50,68.50,46.9577,7976900,0.0,1.0,WMT,4,Friday,January,NYSE,2000,1
41633,2000-01-06,63.0000,64.56,62.69,63.69,43.6604,6544500,0.0,1.0,WMT,3,Thursday,January,NYSE,2000,1
41634,2000-01-05,63.8800,64.38,62.19,62.81,43.0571,7018700,0.0,1.0,WMT,2,Wednesday,January,NYSE,2000,1
41635,2000-01-04,65.5000,65.81,64.19,64.31,44.0854,6745100,0.0,1.0,WMT,1,Tuesday,January,NYSE,2000,1


## Dimension and Fact table creation

In [62]:
dimDate = df_combine[['timestamp', 'day_name', 'day_of_week', 'monthName', 'month', 'year']]

In [63]:
dimDate

,timestamp,day_name,day_of_week,monthName,month,year
0,2020-09-01,Tuesday,1,September,9,2020
1,2020-08-31,Monday,0,August,8,2020
2,2020-08-28,Friday,4,August,8,2020
3,2020-08-27,Thursday,3,August,8,2020
4,2020-08-26,Wednesday,2,August,8,2020
...,...,...,...,...,...,...
41632,2000-01-07,Friday,4,January,1,2000
41633,2000-01-06,Thursday,3,January,1,2000
41634,2000-01-05,Wednesday,2,January,1,2000
41635,2000-01-04,Tuesday,1,January,1,2000


In [64]:
dimSymbol = df_combine[['Symbol', 'exchange']]

In [65]:
dimSymbol = dimSymbol.drop_duplicates()

In [66]:
dimSymbol

,Symbol,exchange
0,AAPL,NASDAQ
5200,AMZN,NASDAQ
10400,BRK-A,NYSE
15600,FB,NASDAQ
17686,GOOG,NASDAQ
19307,JNJ,NYSE
24507,MA,NYSE
28100,MSFT,NASDAQ
33300,V,NYSE
36437,WMT,NYSE


In [67]:
factStock = df_combine[['timestamp', 'Symbol', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']]

In [68]:
factStock

,timestamp,Symbol,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2020-09-01,AAPL,132.7600,134.80,130.53,134.18,134.1800,152470142,0.0,1.0
1,2020-08-31,AAPL,127.5800,131.00,126.00,129.04,129.0400,223505733,0.0,4.0
2,2020-08-28,AAPL,504.0500,505.77,498.31,499.23,124.8075,46907479,0.0,1.0
3,2020-08-27,AAPL,508.5700,509.94,495.33,500.04,125.0100,38888096,0.0,1.0
4,2020-08-26,AAPL,504.7165,507.97,500.33,506.09,126.5225,40755567,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
41632,2000-01-07,WMT,64.5000,68.94,64.50,68.50,46.9577,7976900,0.0,1.0
41633,2000-01-06,WMT,63.0000,64.56,62.69,63.69,43.6604,6544500,0.0,1.0
41634,2000-01-05,WMT,63.8800,64.38,62.19,62.81,43.0571,7018700,0.0,1.0
41635,2000-01-04,WMT,65.5000,65.81,64.19,64.31,44.0854,6745100,0.0,1.0


## Writing the files to S3

In [194]:
csv_buffer = StringIO()

In [182]:
factStock.to_csv(csv_buffer)

In [185]:
s3_resource = boto3.resource('s3',
                    region_name = 'us-east-1',
                    aws_access_key_id=AWS_ACCESS_KEY,
                    aws_secret_access_key=AWS_SECRET_KEY,)
s3_resource.Object(S3_BUCKET_NAME, 'financialanalysis/factStock.csv').put(Body = csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'TQ16J4BEECBZE522',
  'HostId': 'TsxcO2mJT4KvMy5rEbeXN13U9KnlTMvV8UmIsyCV9rEm7dSxxcZFJABKTHKAwlGzAwW8e1Pfx8M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TsxcO2mJT4KvMy5rEbeXN13U9KnlTMvV8UmIsyCV9rEm7dSxxcZFJABKTHKAwlGzAwW8e1Pfx8M=',
   'x-amz-request-id': 'TQ16J4BEECBZE522',
   'date': 'Tue, 13 Feb 2024 02:25:12 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d8b96dfc95c8101081e60bbfb6200719"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d8b96dfc95c8101081e60bbfb6200719"',
 'ServerSideEncryption': 'AES256'}

In [191]:
dimDate.to_csv(csv_buffer)

In [193]:
s3_resource.Object(S3_BUCKET_NAME, 'financialanalysis/dimDate.csv').put(Body = csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '6MMPCR1Z9F39QW3V',
  'HostId': 'htP0H1mX+bRoZmREBZzM3eN2Lw4/dI47Q9Zgd93viVgG0VU0Wc1uIbmMtBuz5s/ew7aBgOZqOqk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'htP0H1mX+bRoZmREBZzM3eN2Lw4/dI47Q9Zgd93viVgG0VU0Wc1uIbmMtBuz5s/ew7aBgOZqOqk=',
   'x-amz-request-id': '6MMPCR1Z9F39QW3V',
   'date': 'Tue, 13 Feb 2024 02:25:52 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3ac3ca90d0923c0719f09a865b9a158e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3ac3ca90d0923c0719f09a865b9a158e"',
 'ServerSideEncryption': 'AES256'}

In [195]:
dimSymbol.to_csv(csv_buffer)

In [197]:
s3_resource.Object(S3_BUCKET_NAME, 'financialanalysis/dimSymbol.csv').put(Body = csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'T586ZPFTCPSKBCPV',
  'HostId': 'ZN1A5TNaNPdXAMVDX8Ss1u5h+bhkcO1NPYQowdzByc6Rk21r3kam8iNYSS5yantrDV/NWzVfu7Q=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZN1A5TNaNPdXAMVDX8Ss1u5h+bhkcO1NPYQowdzByc6Rk21r3kam8iNYSS5yantrDV/NWzVfu7Q=',
   'x-amz-request-id': 'T586ZPFTCPSKBCPV',
   'date': 'Tue, 13 Feb 2024 02:26:07 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5e97d9f04b23374dae8e1cc3ea23050b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5e97d9f04b23374dae8e1cc3ea23050b"',
 'ServerSideEncryption': 'AES256'}

In [196]:
csv_buffer.getvalue()

',Symbol,exchange\r\n0,AAPL,NASDAQ\r\n5200,AMZN,NASDAQ\r\n10400,BRK-A,NYSE\r\n15600,FB,NASDAQ\r\n17686,GOOG,NASDAQ\r\n19307,JNJ,NYSE\r\n24507,MA,NYSE\r\n28100,MSFT,NASDAQ\r\n33300,V,NYSE\r\n36437,WMT,NYSE\r\n'

## Getting SQL Schema

In [70]:
dimDateSql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
print(''.join(dimDateSql))

CREATE TABLE "dimDate" (
"index" INTEGER,
  "timestamp" TIMESTAMP,
  "day_name" TEXT,
  "day_of_week" INTEGER,
  "monthName" TEXT,
  "month" INTEGER,
  "year" INTEGER
)


In [71]:
dimSymbolSql = pd.io.sql.get_schema(dimSymbol.reset_index(), 'dimSymbol')
print(''.join(dimSymbolSql))

CREATE TABLE "dimSymbol" (
"index" INTEGER,
  "Symbol" TEXT,
  "exchange" TEXT
)


In [73]:
factStockSql = pd.io.sql.get_schema(factStock.reset_index(), 'faceStock')
print(''.join(factStockSql))

CREATE TABLE "faceStock" (
"index" INTEGER,
  "timestamp" TIMESTAMP,
  "Symbol" TEXT,
  "open" REAL,
  "high" REAL,
  "low" REAL,
  "close" REAL,
  "adjusted_close" REAL,
  "volume" INTEGER,
  "dividend_amount" REAL,
  "split_coefficient" REAL
)


## Connecting Redshift with Python

In [74]:
import redshift_connector

In [78]:
conn = redshift_connector.connect(
    host = 'default-workgroup2.992382380009.us-east-1.redshift-serverless.amazonaws.com',
    database = 'dev',
    user = 'arvind-serverless',
    password = 'Insomnious!220822'
)

In [81]:
conn.autocommit = True

In [82]:
cursor = redshift_connector.Cursor = conn.cursor()

In [ ]:
cursor.execute("""
   CREATE TABLE "dimDate" (
"index" INTEGER,
  "timestamp" TIMESTAMP,
  "day_name" TEXT,
  "day_of_week" INTEGER,
  "monthName" TEXT,
  "month" INTEGER,
  "year" INTEGER
) 
""")

cursor.execute("""
CREATE TABLE "dimSymbol" (
"index" INTEGER,
  "Symbol" TEXT,
  "exchange" TEXT
)
""")

cursor.execute("""
CREATE TABLE "faceStock" (
"index" INTEGER,
  "timestamp" TIMESTAMP,
  "Symbol" TEXT,
  "open" REAL,
  "high" REAL,
  "low" REAL,
  "close" REAL,
  "adjusted_close" REAL,
  "volume" INTEGER,
  "dividend_amount" REAL,
  "split_coefficient" REAL
)
""")

In [ ]:
cursor.execute("""
    copy dimDate from 's3://financialanalysis/financialanalysis/dimDate.csv'
    credentials 'aws_iam_role=arn:aws:iam::992382380009:role/redshift-s3-access'
    delimiter ','
    region 'us-east-1'
    IGNOREHEADER 1
""")